In [1]:
#Step 0. 필요한 모듈과 라이브러리를 로딩합니다.
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리
from datetime import datetime
from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
from selenium.webdriver import ActionChains
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 크롬 웹브라우저 실행
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)
# 페이지 접속
driver.get('https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000158579&dispCatNo=90000010009&trackingCd=Best_Sellingbest')
time.sleep(2)  # 2초간 정지

In [ ]:
# id가 something 인 element 를 찾음
some_tag = driver.find_element_by_css_selector('.goods_reputation')

# somthing element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(some_tag).perform()
time.sleep(2)

In [ ]:
review_click = driver.find_element_by_id('reviewInfo').click()
time.sleep(2)

In [ ]:
os.makedirs('result', exist_ok=True)

In [ ]:
# create empty dataframe to store data
df = pd.DataFrame(columns=['comment','rate'])


# click 횟수
click_number = 12

for i in range(2,click_number):
#try :
    # 리뷰내용 추출
    comment = driver.find_elements_by_css_selector('.txt_inner')
    comment_list= []
    comment = driver.find_elements_by_css_selector('.txt_inner')
    for m in range(len(comment)):
        comment_list.append(comment[m].text) 
    # 평점 추출    
    rate = driver.find_elements_by_css_selector('.point')  
    rate_list= []
    rate = driver.find_elements_by_css_selector('.point')
    for n in range(len(rate)):
        rate_list.append(rate[n].text) 
    time.sleep(2)
    # append to dataframe
    for j in range(len(comment)):
        df = df.append({'rate': rate_list[j+1] ,'comment': comment_list[j]}, ignore_index=True)
    print(f'{i-1}페이지  크롤링중')

    some_tag = driver.find_element_by_css_selector(f'#gdasContentsArea > div > div.pageing > a:nth-child({i})')
    # somthing element 까지 스크롤
    action = ActionChains(driver)
    action.move_to_element(some_tag).perform()
    time.sleep(3)
    page = driver.find_element_by_css_selector(f'#gdasContentsArea > div > div.pageing > a:nth-child({i})').click()
    time.sleep(2)         
#except:
    #print('error')

# finally save the dataframe into csv file
filename = datetime.now().strftime('result/%Y-%m-%d_%H-%M-%S.csv')
df.to_csv(filename, encoding='utf-8-sig', index=False)
driver.stop_client()
driver.close()

print('Done!')